<a href="https://colab.research.google.com/github/aviax1/AE1/blob/master/improve_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**dependencies**

In [41]:
import wandb
import torch.nn as nn
hyperparameter_defaults = dict(
    dropout = 0.5,
    hd1 = 16,
    lv1 = 32,
    bt = 100,
    learning_rate = 0.001,
    epochs = 200,
)
wandb.init(config=hyperparameter_defaults, project="ae1")
!wandb agent aviax1/ae1/yjyslrpc